<a href="https://colab.research.google.com/github/svetlanama/math/blob/main/%D0%94%D0%9710_%D0%9C%D0%BE%D1%96%D1%81%D0%B5%D1%94%D0%BD%D0%BA%D0%BE_%D0%A1%D0%B2%D1%96%D1%82%D0%BB%D0%B0%D0%BD%D0%B0_%D0%9E%D0%BB%D0%B5%D0%BA%D1%81%D0%B0%D0%BD%D0%B4%D1%80%D1%96%D0%B2%D0%BD%D0%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# =========================
# PART 1: ANALYSIS & PREP
# Dataset: Kaggle "House Rent Prediction Dataset"
# Link: https://www.kaggle.com/datasets/iamsouravbanerjee/house-rent-prediction-dataset/data
# =========================

# ---------- (1) Kaggle download (Colab-friendly) ----------
import os, json, zipfile, glob, shutil
from pathlib import Path

IN_COLAB = "COLAB_GPU" in os.environ or "COLAB_BACKEND_VERSION" in os.environ

if IN_COLAB:
    # 1) Встановити kaggle-cli
    !pip -q install kaggle >/dev/null

    # 2) Створити папку .kaggle (якщо немає)
    Path.home().joinpath(".kaggle").mkdir(exist_ok=True)

    # 3) Завантажити kaggle.json (через GUI: Files → Upload)
    # Якщо вже завантажили в /content, перенесемо:
    if Path("/content/kaggle.json").exists():
        shutil.move("/content/kaggle.json", str(Path.home().joinpath(".kaggle/kaggle.json")))

    # 4) Права доступу
    kaggle_json = Path.home().joinpath(".kaggle/kaggle.json")
    if kaggle_json.exists():
        kaggle_json.chmod(0o600)
    else:
        print("⚠️ Завантажте спершу kaggle.json у /content або покладіть у ~/.kaggle/")

    # 5) Завантаження датасету
    !kaggle datasets download -d iamsouravbanerjee/house-rent-prediction-dataset -p /content/data -o
    # Розпакування
    Path("/content/data").mkdir(exist_ok=True, parents=True)
    for zf in glob.glob("/content/data/*.zip"):
        with zipfile.ZipFile(zf, "r") as z:
            z.extractall("/content/data")
else:
    print("ℹ️ Не Colab середовище. Переконайтесь, що датасет завантажений локально або встановіть kaggle-cli вручну.")

# ---------- (2) Load data ----------
import pandas as pd
import numpy as np

# Спробуємо знайти CSV, якщо у Colab зкачали — він лежить в /content/data
possible_paths = [
    "/content/data/House_Rent_Dataset.csv",
    "House_Rent_Dataset.csv",
]

csv_path = None
for p in possible_paths:
    if Path(p).exists():
        csv_path = p
        break

if csv_path is None:
    # fallback: спроба знайти будь-який CSV у /content/data
    candidates = glob.glob("/content/data/*.csv")
    if candidates:
        csv_path = candidates[0]

if csv_path is None:
    raise FileNotFoundError("Не знайдено CSV. Переконайтесь, що датасет завантажено. Очікував House_Rent_Dataset.csv")

df = pd.read_csv(csv_path)

print("✅ Завантажено:", csv_path)
print("Форма:", df.shape)
display(df.head(3))
display(df.tail(3))

# ---------- (3) Базова інформація: типи, описова статистика ----------
print("\n=== INFO ===")
print(df.info())

print("\n=== Описова статистика (числові) ===")
display(df.describe().T)

print("\n=== Описова статистика (всі стовпці) ===")
display(df.describe(include="all").T)

# ---------- (4) Перевірка пропусків ----------
print("\n=== Пропущені значення (кількість та частка) ===")
na_counts = df.isna().sum().sort_values(ascending=False)
na_share = (df.isna().mean()*100).sort_values(ascending=False)
display(pd.DataFrame({"na_count": na_counts, "na_%": na_share}).query("na_count > 0"))

# ---------- (5) Аналіз типів ознак ----------
numeric_cols = df.select_dtypes(include=[np]()


SyntaxError: incomplete input (ipython-input-2136941612.py, line 91)